In [1]:
sc.stop()

NameError: name 'sc' is not defined

In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
!hdfs dfs -ls /labs/slaba02/DO_record_per_line.json

-rw-r--r--   3 hdfs hdfs   69519728 2021-02-27 21:58 /labs/slaba02/DO_record_per_line.json


In [3]:
from pyspark.sql.types import LongType, StringType, DoubleType, StructType, StructField, IntegerType

In [4]:
schema = StructType(fields=[
    StructField("cat", StringType()),
    StructField("desc", StringType()),
    StructField("id", IntegerType()),
    StructField("lang", StringType()),
    StructField("name", StringType()),
    StructField("provider", StringType())
])

In [5]:
df = sqlContext.read.json('/labs/slaba02/DO_record_per_line.json', schema=schema)
df.show()

+--------------------+--------------------+---+----+--------------------+--------------+
|                 cat|                desc| id|lang|                name|      provider|
+--------------------+--------------------+---+----+--------------------+--------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|
|9/humanities|15/m...|This game-based c...|  9|  en|College Foundatio...|Canvas Network|
|  14/social_sciences|What’s in your di...| 10|  en|Digital Literacies I|Canvas Network|
|  14/social_sciences|The goal of the D...| 11|  en|Digital Literacie...|Canvas Network|
|  14/social_sciences

In [6]:
df.printSchema()

root
 |-- cat: string (nullable = true)
 |-- desc: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- lang: string (nullable = true)
 |-- name: string (nullable = true)
 |-- provider: string (nullable = true)



In [7]:
from pyspark.sql.functions import lower, col
import re
regex = re.compile(u'[\w\d]{2,}', re.U)
string = df.take(1)[0].name
regex.findall(string.lower())[0]

'accounting'

In [8]:
regex.findall(df.take(1)[0].name.lower())

['accounting',
 'cycle',
 'the',
 'foundation',
 'of',
 'business',
 'measurement',
 'and',
 'reporting']

In [14]:
from pyspark.sql.functions import udf
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
from pyspark.sql.types import LongType, ArrayType, StringType
from pyspark.sql.functions import lower, col
import re

regex = re.compile(u'[\w\d]{2,}', re.U)
string=df.take(1)[0].name
regex.findall(string.lower())

def ppp2(string):
     return regex.findall(string.lower())
    

print(ppp2(df.take(1)[0].desc))

['this', 'course', 'introduces', 'the', 'basic', 'financial', 'statements', 'used', 'by', 'most', 'businesses', 'as', 'well', 'as', 'the', 'essential', 'tools', 'used', 'to', 'prepare', 'them', 'this', 'course', 'will', 'serve', 'as', 'resource', 'to', 'help', 'business', 'students', 'succeed', 'in', 'their', 'upcoming', 'university', 'level', 'accounting', 'classes', 'and', 'as', 'refresher', 'for', 'upper', 'division', 'accounting', 'students', 'who', 'are', 'struggling', 'to', 'recall', 'elementary', 'concepts', 'essential', 'to', 'more', 'advanced', 'accounting', 'topics', 'business', 'owners', 'will', 'also', 'benefit', 'from', 'this', 'class', 'by', 'gaining', 'essential', 'skills', 'necessary', 'to', 'organize', 'and', 'manage', 'information', 'pertinent', 'to', 'operating', 'their', 'business', 'at', 'the', 'conclusion', 'of', 'the', 'class', 'students', 'will', 'understand', 'the', 'balance', 'sheet', 'income', 'statement', 'and', 'cash', 'flow', 'statement', 'they', 'will', '

In [15]:
my_udfi = udf(ppp2, returnType=StringType())

In [16]:
my_courses = [[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']]
for i in my_courses:
    print(i[0])

23126
21617
16627
11556
16704
13702


In [17]:
my_cours = df.where('id in (23126, 21617, 16627, 11556, 16704, 13702)')
my_cours.show()

+--------------------+--------------------+-----+----+--------------------+--------+
|                 cat|                desc|   id|lang|                name|provider|
+--------------------+--------------------+-----+----+--------------------+--------+
|                    | La transformació...|11556|  es|Aprendizaje Colab...|   Udemy|
|6/economics_finan...|Математическая эк...|13702|  ru|Математическая эк...|  Intuit|
|                    | Hazte más emplea...|16627|  es|Aprende Excel: Ni...|   Udemy|
|5/computer_scienc...|В курсе рассматри...|16704|  ru|Программирование ...|  Intuit|
|  5/computer_science|An introduction t...|21617|  en|Preparing for the...|     edX|
|                    | Improve your SAS...|23126|  en|Compass - powerfu...|   Udemy|
+--------------------+--------------------+-----+----+--------------------+--------+



In [18]:
my_cours.count()

6

In [19]:
from pyspark.sql.functions import split

In [20]:
from pyspark.ml.linalg import DenseVector, SparseVector

In [21]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

In [22]:
import pyspark.sql.functions as f

In [24]:
tokenizer = Tokenizer(inputCol="desc", outputCol="words")
wordsData = tokenizer.transform(df)

In [25]:
wordsData.show(5)

+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|                 cat|                desc| id|lang|                name|      provider|               words|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
|3/business_manage...|This course intro...|  4|  en|Accounting Cycle:...|Canvas Network|[this, course, in...|
|              11/law|This online cours...|  5|  en|American Counter ...|Canvas Network|[this, online, co...|
|5/computer_scienc...|This course is ta...|  6|  fr|Arithmétique: en ...|Canvas Network|[this, course, is...|
|  14/social_sciences|We live in a digi...|  7|  en|Becoming a Dynami...|Canvas Network|[we, live, in, a,...|
|2/biology_life_sc...|This self-paced c...|  8|  en|           Bioethics|Canvas Network|[this, self-paced...|
+--------------------+--------------------+---+----+--------------------+--------------+--------------------+
only showi

In [26]:
wordsData.count()

28153

In [27]:
wordsData=wordsData.withColumn("count", f.size("words")).where("count<>0")
wordsData.count()

28125

In [28]:
hashingTF = HashingTF(inputCol="words", outputCol="hTF", numFeatures=10000)
featurizedData = hashingTF.transform(wordsData)

idf = IDF(inputCol="hTF", outputCol="TFIDF")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)

### Тест

In [29]:
schema2=df.select("id","lang","desc").schema

In [47]:
_Base = spark.createDataFrame([[23126, u'en', u'Compass - powerful SASS library that makes your life easier'], [21617, u'en', u'Preparing for the AP* Computer Science A Exam \u2014 Part 2'], [16627, u'es', u'Aprende Excel: Nivel Intermedio by Alfonso Rinsche'], [11556, u'es', u'Aprendizaje Colaborativo by UNID Universidad Interamericana para el Desarrollo'], [16704, u'ru', u'\u041f\u0440\u043e\u0433\u0440\u0430\u043c\u043c\u0438\u0440\u043e\u0432\u0430\u043d\u0438\u0435 \u043d\u0430 Lazarus'], [13702, u'ru', u'\u041c\u0430\u0442\u0435\u043c\u0430\u0442\u0438\u0447\u0435\u0441\u043a\u0430\u044f \u044d\u043a\u043e\u043d\u043e\u043c\u0438\u043a\u0430']],schema=schema2)

In [48]:
_join=_Base.select("id").join(rescaledData.select("id","lang","TFIDF"),"id", how="inner").filter(col('id').isin([23126, 21617, 16627, 11556, 16704, 13702]))

In [49]:
_join.show()

+-----+----+--------------------+
|   id|lang|               TFIDF|
+-----+----+--------------------+
|13702|  ru|(10000,[310,763,9...|
|16627|  es|(10000,[30,145,19...|
|16704|  ru|(10000,[381,1144,...|
|23126|  en|(10000,[87,91,96,...|
|21617|  en|(10000,[213,360,4...|
|11556|  es|(10000,[249,522,5...|
+-----+----+--------------------+



In [50]:
join=_join.join(rescaledData.select(col("id").alias("_id"),"lang",col("TFIDF").alias("_TFIDF")),"lang", how="inner")

In [51]:
@f.udf('double')
def cos(x,y):
    return float(x.dot(y) / (x.norm(2) * y.norm(2)))

In [52]:
res=join.withColumn('cos',cos("TFIDF","_TFIDF"))

In [53]:
res.where("id=_id").orderBy("cos", ascending=False).show()

+----+-----+--------------------+-----+--------------------+------------------+
|lang|   id|               TFIDF|  _id|              _TFIDF|               cos|
+----+-----+--------------------+-----+--------------------+------------------+
|  en|23126|(10000,[87,91,96,...|23126|(10000,[87,91,96,...|1.0000000000000002|
|  es|16627|(10000,[30,145,19...|16627|(10000,[30,145,19...|1.0000000000000002|
|  ru|13702|(10000,[310,763,9...|13702|(10000,[310,763,9...|1.0000000000000002|
|  ru|16704|(10000,[381,1144,...|16704|(10000,[381,1144,...|               1.0|
|  es|11556|(10000,[249,522,5...|11556|(10000,[249,522,5...|               1.0|
|  en|21617|(10000,[213,360,4...|21617|(10000,[213,360,4...|0.9999999999999999|
+----+-----+--------------------+-----+--------------------+------------------+



In [54]:
res=res.select("lang","id","_id","TFIDF","_TFIDF","cos").where("id<>_id").orderBy("id","cos", ascending=False)

In [55]:
res.count()

54256

In [56]:
from pyspark.sql import Window

In [61]:
res_out=res.select("id","_id","cos", f.row_number().over(Window.partitionBy("id").orderBy(col("cos").desc())).alias("N"))

In [62]:
res_out=res_out.where("N<=10")

In [63]:
res_out.where("id=16704").show()

+-----+-----+-------------------+---+
|   id|  _id|                cos|  N|
+-----+-----+-------------------+---+
|16704| 1365|0.30737172340483715|  1|
|16704|20645|0.29816668697589227|  2|
|16704|20105|  0.296335695787284|  3|
|16704| 1426|  0.296335695787284|  4|
|16704| 8217|0.28745405355016035|  5|
|16704| 1236| 0.2865114161415227|  6|
|16704| 1164| 0.2823455375759388|  7|
|16704| 1219|0.27430304672152866|  8|
|16704| 8123|0.27147713280944274|  9|
|16704|  875|0.27119516453913817| 10|
+-----+-----+-------------------+---+



In [64]:
res_out.where("id=16704").select("_id").show()

+-----+
|  _id|
+-----+
| 1365|
|20645|
|20105|
| 1426|
| 8217|
| 1236|
| 1164|
| 1219|
| 8123|
|  875|
+-----+



In [67]:
res_out.groupBy('id')

AttributeError: 'GroupedData' object has no attribute 'flatMap'

In [70]:
import pyspark.sql.functions as F

In [72]:
res_out.cache()

DataFrame[id: int, _id: int, cos: double, N: int]

In [74]:
list_res = res_out.groupby(col('id')).agg(F.collect_list(col('_id')).alias('merged')).take(6)

In [78]:
list_res[0][1]

[864, 1216, 7173, 8313, 1052, 17017, 19613, 21017, 17015, 8082]

In [80]:
final_json = {}
for elem in list_res:
    final_json[elem[0]] = elem[1]

In [81]:
final_json

{13702: [864, 1216, 7173, 8313, 1052, 17017, 19613, 21017, 17015, 8082],
 16627: [11431, 12247, 13021, 25010, 11575, 5687, 9598, 5372, 12863, 16769],
 16704: [1365, 20645, 1426, 20105, 8217, 1236, 1164, 1219, 8123, 875],
 23126: [13782, 13665, 24419, 20638, 2724, 25782, 2633, 2723, 13348, 15909],
 21617: [21609, 21608, 21616, 21492, 21624, 21623, 21630, 21628, 21508, 21857],
 11556: [16488, 13461, 468, 10447, 387, 22710, 9289, 5936, 23357, 7833]}

In [ ]:
import json

In [83]:
with open('lab02.json', 'w') as f:
    json.dump(final_json, f)